The workshop contains three different notebooks. Each one focuses on a different stage:
    
1. Dataset Generation. The first notebook (this one) focuses on generating a dataset for training the model. We will create a Robust Test Suite to check that the dataset generated meets certain conditions
2. Model Training. The second notebook focuses on training the model. We will create a Robust Test Suite to check that the trained model meets certain conditions.
3. Model Inference. In the last notebook, we use mercury.monitoring to monitor data drift and estimate the predicted performance of the model without having the labels

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(seed=2021)

pd.set_option('display.max_colwidth', None)

SEED = 42

## Load Dataset

We will use the default credit card Dataset from the UCI machine learning repository. The dataset was used in [[1]](#[1]). Note that we will use a slightly modified version which contains a time column

In [2]:
df = pd.read_csv("data/uci_credit_drifted_historic.csv")

In [3]:
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month,REMAINDER_SENT,time,id
0,50000.0,1,2,1,57,-1,0,-1,0,0,...,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,0,1,4
1,500000.0,1,1,2,29,0,0,0,0,0,...,55000.0,40000.0,38000.0,20239.0,13750.0,13770.0,0,0,1,6
2,100000.0,2,2,2,23,0,-1,-1,0,0,...,380.0,601.0,0.0,581.0,1687.0,1542.0,0,0,1,7
3,260000.0,2,1,2,51,-1,-1,-1,-1,-1,...,21818.0,9966.0,8583.0,22301.0,0.0,3640.0,0,0,1,11
4,630000.0,2,2,2,41,-1,0,-1,-1,-1,...,1000.0,6500.0,6500.0,6500.0,2870.0,0.0,0,0,1,12


## Prepare Dataset For Training

Let's preparing the dataset for training the model. Our label will be the "default.payment.next.month" variable. We select the features that we want to use in our model:

In [4]:
label = 'default.payment.next.month'
#features = [c for c in df.columns if c not in [label, 'time', 'PAYMENT_DELAY', 'id']]
features = [c for c in df.columns if c not in [label, 'time', 'PAYMENT_DELAY', 'id', 'REMAINDER_SENT']]

In [5]:
features

['LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6']

Now, let's define the function that will generate a train and test dataset:

In [6]:
from sklearn.model_selection import train_test_split

def prepare_dataset(df, features, label, test_size=0.3, random_state=42):
    
    # Only Keep Features and label
    df = df[features + [label]]
    
    # Drop Duplicates
    df = df.drop_duplicates()
    
    # Split Train/Test
    df_train, df_test = train_test_split(df, test_size=test_size, random_state=random_state)
    
    return df, df_train, df_test

In [7]:
df, df_train, df_test = prepare_dataset(df, features, label, test_size=0.3, random_state=SEED)

print(df_train.shape)
print(df_test.shape)

(7135, 24)
(3059, 24)


## Create Data Schema

We now use [mercury.dataschema](https://bbva.github.io/mercury-dataschema/) to create a `DataSchema` which contains the feature types of the dataset. This will be used later when creating the Robust Tests. 

The `DataSchema` auto-infers the feature types, but we can also specify some feature types in case that the auto-inference doesn't work exactly as we want.

In [8]:
from mercury.dataschema import DataSchema
from mercury.dataschema.feature import FeatType

custom_feature_mapping = {
    "PAY_0": FeatType.DISCRETE,
    "PAY_2": FeatType.DISCRETE,
    "PAY_3": FeatType.DISCRETE,
    "PAY_4": FeatType.DISCRETE,
    "PAY_5": FeatType.DISCRETE,
    "PAY_6": FeatType.DISCRETE,
}

schema = DataSchema().generate(df_train, force_types=custom_feature_mapping).calculate_statistics()

In [9]:
schema.feats

{'LIMIT_BAL': Discrete Feature (NAME=LIMIT_BAL, dtype=DataType.FLOAT),
 'SEX': Binary Feature (NAME=SEX, dtype=DataType.INTEGER),
 'EDUCATION': Categorical Feature (NAME=EDUCATION, dtype=DataType.INTEGER),
 'MARRIAGE': Categorical Feature (NAME=MARRIAGE, dtype=DataType.INTEGER),
 'AGE': Discrete Feature (NAME=AGE, dtype=DataType.INTEGER),
 'PAY_0': Discrete Feature (NAME=PAY_0, dtype=DataType.INTEGER),
 'PAY_2': Discrete Feature (NAME=PAY_2, dtype=DataType.INTEGER),
 'PAY_3': Discrete Feature (NAME=PAY_3, dtype=DataType.INTEGER),
 'PAY_4': Discrete Feature (NAME=PAY_4, dtype=DataType.INTEGER),
 'PAY_5': Discrete Feature (NAME=PAY_5, dtype=DataType.INTEGER),
 'PAY_6': Discrete Feature (NAME=PAY_6, dtype=DataType.INTEGER),
 'BILL_AMT1': Discrete Feature (NAME=BILL_AMT1, dtype=DataType.FLOAT),
 'BILL_AMT2': Discrete Feature (NAME=BILL_AMT2, dtype=DataType.FLOAT),
 'BILL_AMT3': Discrete Feature (NAME=BILL_AMT3, dtype=DataType.FLOAT),
 'BILL_AMT4': Discrete Feature (NAME=BILL_AMT4, dtype=Da

## Data Robust Tests

We now [mercury.robust](https://bbva.github.io/mercury-robust/) to create tests to check that the generated dataset meets certain conditions.

More concretely, we will create the next tests:
1. [LinearCombinationsTest](https://bbva.github.io/mercury-robust/reference/data_tests/#mercury.robust.data_tests.LinearCombinationsTest): Ensures that the dataset doesn't have any linear combination between its numerical columns and no categorical variable is redundant
2. [LabelLeakingTest](https://bbva.github.io/mercury-robust/reference/data_tests/#mercury.robust.data_tests.LabelLeakingTest): Ensures the target variable is not being leaked into the predictors.
3. [NoisyLabelTest](https://bbva.github.io/mercury-robust/reference/data_tests/#mercury.robust.data_tests.NoisyLabelsTest): Looks if the labels of a dataset contain a high level of noise.
4. [SampleLeakingTest](https://bbva.github.io/mercury-robust/reference/data_tests/#mercury.robust.data_tests.SampleLeakingTest): Looks if there are samples in the test dataset that are identical to samples in the base/train dataset.
5. [NoDuplicatesTest](https://bbva.github.io/mercury-robust/reference/data_tests/#mercury.robust.data_tests.NoDuplicatesTest): Checks no duplicated samples are present in a dataframe

In [10]:
from mercury.robust.data_tests import (
    LinearCombinationsTest,
    LabelLeakingTest,
    NoisyLabelsTest,
    SampleLeakingTest,
    NoDuplicatesTest
)

We have two options to execute the tests: We can just execute one test individually, or alternatively, run a group of test in a `TestSuite`.

Let's start running an individual test:

In [11]:
# LinearCombinationsTest
linear_combinations = LinearCombinationsTest(df[features], dataset_schema=schema)
linear_combinations.run()

When no exception is raised, the test has run successfully. Let's try another test:

In [12]:
# LabelLeakingTest
label_leaking = LabelLeakingTest(
    df[features + [label]], 
    label_name = label,
    task = "classification",
    dataset_schema=schema,
)
label_leaking.run()

Now the test has failed

## Test Suite

Now we will group several test in a `TestSuite` and execute them together.

In [13]:
from mercury.robust import TestSuite

def create_suite(df, df_train, df_test, schema, features, label):

    # LinearCombinationsTest
    linear_combinations = LinearCombinationsTest(df[features], dataset_schema=schema)
    
    # LabelLeakingTest
    label_leaking = LabelLeakingTest(
        df[features + [label]], 
        label_name = label,
        task = "classification",
        dataset_schema=schema,
    )
    
    # Noisy Labels
    noisy_labels = NoisyLabelsTest(
        base_dataset=df[features + [label]],
        label_name=label,
        calculate_idx_issues=True,
        threshold = 0.2,
        dataset_schema=schema,
        label_issues_args={"clf": LogisticRegression(solver='liblinear')}
    )
    
    # SampleLeaking
    sample_leaking = SampleLeakingTest(
        base_dataset=df_train[features + [label]], 
        test_dataset=df_test[features + [label]]
    )
    
    # NoDuplicates
    no_dups = NoDuplicatesTest(df_train)
    
    # Create Suite
    test_suite = TestSuite(
        tests=[
            linear_combinations,
            label_leaking,
            noisy_labels,
            sample_leaking,
            no_dups
        ]
    )
    
    return test_suite


In [14]:
test_suite = create_suite(df, df_train, df_test, schema, features, label)
test_results = test_suite.run()
test_suite.get_results_as_df()

,name,state,error,info
0,LinearCombinationsTest,TestState.SUCCESS,,None
1,LabelLeakingTest,TestState.SUCCESS,,"{'importances': {'LIMIT_BAL': 0.5, 'SEX': 0.5, 'EDUCATION': 0.5, 'MARRIAGE': 0.5, 'PAY_AMT2': 0.5, 'BILL_AMT5': 0.5, 'PAY_AMT4': 0.5003431708991077, 'PAY_AMT6': 0.5003431708991077, 'PAY_AMT5': 0.5006291139167917, 'PAY_AMT3': 0.5006291139167917, 'AGE': 0.5006291139167917, 'PAY_6': 0.5006863417982155, 'BILL_AMT2': 0.5006863417982155, 'BILL_AMT6': 0.5006863417982155, 'PAY_AMT1': 0.5006863417982155, 'PAY_4': 0.5009150569344757, 'BILL_AMT4': 0.5009722848158995, 'BILL_AMT1': 0.5010295126973233, 'BILL_AMT3': 0.5017158544955388, 'PAY_3': 0.5029166580582631, 'PAY_5': 0.503202601075947, 'PAY_2': 0.5052043985891697, 'PAY_0': 0.5187019302489142}}"
2,NoisyLabelsTest,TestState.SUCCESS,,{'rate_issues': 0.06533254855797528}
3,SampleLeakingTest,TestState.SUCCESS,,"{'num_duplicated': 0, 'percentage_duplicated': 0.0}"
4,NoDuplicatesTest,TestState.SUCCESS,,"{'num_duplicated': 0, 'index_duplicates': []}"


## Save Dataset and Data Schema

Let's save our generated dataset and the `DataSchema`

In [15]:
path_dataset = "./dataset/"

if not os.path.exists(path_dataset):
    os.makedirs(path_dataset)

df.to_csv(path_dataset + "all.csv", index=False)
df_train.to_csv(path_dataset + "train.csv", index=False)
df_test.to_csv(path_dataset + "test.csv", index=False)

schema.save(path_dataset + "schema.json")

## References

<a id="[1]">[1]</a>
Yeh, I. C., & Lien, C. H. (2009). The comparisons of data mining techniques for the predictive accuracy of probability of default of credit card clients. Expert Systems with Applications, 36(2), 2473-2480. https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients